In [16]:
from bs4 import BeautifulSoup as bs
import requests
import re
from pprint import pprint

import pandas as pd
import numpy as np

import json

In [17]:
vac = input('Профессия или должность ')

Профессия или должность python


In [18]:
class DataPipeline_sj:
    
    def find_vac(self, vac):
        headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, /'
                            'like Gecko) Chrome/81.0.4044.138 Safari/537.36'}
        
        main_link = 'https://russia.superjob.ru'
        
        vacancies = []
     
        params = {'keywords':vac, 'page':''}

        html = requests.get(main_link + '//vacancy/search/', headers=headers, params=params).text
        soup = bs(html, 'html.parser')

        pager_block = soup.find('div', {'class': '_3zucV L1p51 undefined _2guZ- _GJem'})
        page_n = int(pager_block.find_all('span', {'class': '_3IDf-'})[-2].getText())

        for page in range(1, page_n):
            params = {'keywords': vac, 'page': page}
            html = requests.get(main_link + '//vacancy/search/', headers=headers, params=params).text
            soup = bs(html, 'html.parser')

            vacancy_list = soup.find_all('div', {'class': 'Fo44F QiY08 LvoDO'})

            for vacancy in vacancy_list:
                vacancy_data = {}
                vacancy_name = vacancy.find('a', {'class': 'icMQ_'}).getText()
                vacancy_link = main_link + vacancy.find('a', {'class': 'icMQ_'})['href']
                v_salary = vacancy.find('span',
                                        {'class': 'f-test-text-company-item-salary'}).getText()

                v_salary = v_salary.split()

                if 'По' in v_salary:
                    currency = None
                    salary_min = None
                    salary_max = None
                elif 'от' in v_salary:
                    currency = v_salary[-1]
                    salary_min = int(''.join(re.findall(r'\d', ''.join(v_salary))))
                    salary_max = None
                elif 'до' in v_salary:
                    currency = v_salary[-1]
                    salary_min = None
                    salary_max = int(''.join(re.findall(r'\d', ''.join(v_salary))))
                elif '—' in v_salary:
                    currency = v_salary[-1]
                    v_sal = ''.join(v_salary).split('—')
                    salary_min = int(v_sal[0])
                    salary_max = int(''.join(re.findall(r'\d', ''.join(v_sal[1]))))
                else:
                    currency = v_salary[-1]
                    salary_min = int(''.join(re.findall(r'\d', ''.join(v_salary))))
                    salary_max = int(''.join(re.findall(r'\d', ''.join(v_salary))))


                vacancy_data['name'] = vacancy_name
                vacancy_data['link'] = vacancy_link
                vacancy_data['salary_min'] = salary_min
                vacancy_data['salary_max'] = salary_max
                vacancy_data['currency'] = currency
                vacancy_data['site'] = main_link
                
                vacancies.append(vacancy_data)
                last_page = soup.find('a', {'class': 'f-test-button-dalshe'})
            
            if not last_page:
                break

        return vacancies

In [19]:
vac_data_js = DataPipeline_sj()

In [20]:
vac_data_js = vac_data_js.find_vac(vac)
pprint(vac_data_js)

[{'currency': 'руб.',
  'link': 'https://russia.superjob.ru/vakansii/senior-python-developer-33807534.html',
  'name': 'Senior Python Developer',
  'salary_max': 160000,
  'salary_min': 140000,
  'site': 'https://russia.superjob.ru'},
 {'currency': None,
  'link': 'https://russia.superjob.ru/vakansii/prepodavatel-po-kursu-32448389.html',
  'name': 'Преподаватель по курсу (Python для машинного обучения)',
  'salary_max': None,
  'salary_min': None,
  'site': 'https://russia.superjob.ru'},
 {'currency': 'руб.',
  'link': 'https://russia.superjob.ru/vakansii/sistemnyj-administrator-linux-33832459.html',
  'name': 'Системный администратор Linux',
  'salary_max': None,
  'salary_min': 50000,
  'site': 'https://russia.superjob.ru'},
 {'currency': 'руб.',
  'link': 'https://russia.superjob.ru/vakansii/uchitel-informatiki-32382712.html',
  'name': 'Учитель информатики',
  'salary_max': None,
  'salary_min': 78000,
  'site': 'https://russia.superjob.ru'},
 {'currency': 'руб.',
  'link': 'https:

In [21]:
class DataPipeline:

    def find_vac(self, vac):

        headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)/'
                            ' Chrome/81.0.4044.138 Safari/537.36'}

        vacancies = []

        main_link = 'https://hh.ru/search/vacancy/'

        params = {'text':vac, 'page':''}

        html = requests.get(main_link, params=params, headers=headers).text
        soup = bs(html,'html.parser')

        pager_block = soup.find('div', {'data-qa': 'pager-block'})         
        page_n = int(pager_block.find_all('a', {'class': 'HH-Pager-Control'})[-2].getText())

        for page in range(0, page_n):
            params = {'text': vac, 'page': page}
            html = requests.get(main_link, params=params, headers=headers).text
            soup = bs(html, 'html.parser')

            vacancy_block = soup.find('div', {'class': 'vacancy-serp'})
            vacancy_list = vacancy_block.find_all('div', {'class': 'vacancy-serp-item'})

            for vacancy in vacancy_list:
                vacancy_data = {}
                vacancy_name = vacancy.find('a', {'data-qa': 'vacancy-serp__vacancy-title'}).getText()
                vacancy_link = vacancy.find('a', {'data-qa': 'vacancy-serp__vacancy-title'})['href']
                v_salary = vacancy.find('span', {'data-qa': 'vacancy-serp__vacancy-compensation'})

                if not v_salary:
                    salary_min = None
                    salary_max = None
                    salary_currency = None
                else:
                    v_salary = v_salary.getText().replace(u'\xa0', u'')
                    v_salary = re.split(r'\s|-', v_salary)

                    if v_salary[0] == 'до':
                        salary_min = None
                        salary_max = int(v_salary[1])
                    elif v_salary[0] == 'от':
                        salary_min = int(v_salary[1])
                        salary_max = None
                    else:
                        salary_min = int(v_salary[0])
                        salary_max = int(v_salary[1])

                    salary_currency = v_salary[2]

                vacancy_data['name'] = vacancy_name
                vacancy_data['link'] = vacancy_link
                #vacancy_data['salary'] = v_salary
                vacancy_data['salary_min'] = salary_min
                vacancy_data['salary_max'] = salary_max
                vacancy_data['currency'] = salary_currency
                vacancy_data['site'] = main_link

                vacancies.append(vacancy_data)
                last_page = soup.find('a', {'data-qa':'pager-next'})

            if not last_page:
                break
        
        return vacancies

In [22]:
vac_data = DataPipeline()

In [23]:
vac_data = vac_data.find_vac(vac)
pprint(vac_data )

[{'currency': 'руб.',
  'link': 'https://kaliningrad.hh.ru/vacancy/37022145?query=python',
  'name': 'C# разработчик (начинающий, junior)',
  'salary_max': 140000,
  'salary_min': 100000,
  'site': 'https://hh.ru/search/vacancy/'},
 {'currency': None,
  'link': 'https://kaliningrad.hh.ru/vacancy/35095020?query=python',
  'name': 'Senior Python Developer',
  'salary_max': None,
  'salary_min': None,
  'site': 'https://hh.ru/search/vacancy/'},
 {'currency': None,
  'link': 'https://kaliningrad.hh.ru/vacancy/37079355?query=python',
  'name': 'AI Scientist (R&D)',
  'salary_max': None,
  'salary_min': None,
  'site': 'https://hh.ru/search/vacancy/'},
 {'currency': None,
  'link': 'https://kaliningrad.hh.ru/vacancy/36719374?query=python',
  'name': 'Data Scientist',
  'salary_max': None,
  'salary_min': None,
  'site': 'https://hh.ru/search/vacancy/'},
 {'currency': 'EUR',
  'link': 'https://kaliningrad.hh.ru/vacancy/36357908?query=python',
  'name': 'Rust Developer',
  'salary_max': None,


  'name': 'Python/Django разработчик (part time)',
  'salary_max': 40000,
  'salary_min': 30000,
  'site': 'https://hh.ru/search/vacancy/'},
 {'currency': None,
  'link': 'https://kaliningrad.hh.ru/vacancy/36297018?query=python',
  'name': 'Python developer',
  'salary_max': None,
  'salary_min': None,
  'site': 'https://hh.ru/search/vacancy/'},
 {'currency': 'руб.',
  'link': 'https://kaliningrad.hh.ru/vacancy/36725871?query=python',
  'name': 'Fullstack Web Developer (Vue, Python)',
  'salary_max': 100000,
  'salary_min': 70000,
  'site': 'https://hh.ru/search/vacancy/'},
 {'currency': 'руб.',
  'link': 'https://kaliningrad.hh.ru/vacancy/34649894?query=python',
  'name': 'Full-Stack Web-Developer (Python / Odoo)',
  'salary_max': 200000,
  'salary_min': 100000,
  'site': 'https://hh.ru/search/vacancy/'},
 {'currency': None,
  'link': 'https://kaliningrad.hh.ru/vacancy/36677020?query=python',
  'name': 'Python (Back-end) Developer',
  'salary_max': None,
  'salary_min': None,
  'site'

 {'currency': None,
  'link': 'https://kaliningrad.hh.ru/vacancy/35024187?query=python',
  'name': 'Бэкенд-разработчик',
  'salary_max': None,
  'salary_min': None,
  'site': 'https://hh.ru/search/vacancy/'},
 {'currency': 'руб.',
  'link': 'https://kaliningrad.hh.ru/vacancy/36767801?query=python',
  'name': 'Преподаватель SQL, Java, Big Data',
  'salary_max': None,
  'salary_min': 200000,
  'site': 'https://hh.ru/search/vacancy/'},
 {'currency': None,
  'link': 'https://kaliningrad.hh.ru/vacancy/37045181?query=python',
  'name': 'Data Scientist',
  'salary_max': None,
  'salary_min': None,
  'site': 'https://hh.ru/search/vacancy/'},
 {'currency': 'руб.',
  'link': 'https://kaliningrad.hh.ru/vacancy/36354015?query=python',
  'name': 'Spark data engineer',
  'salary_max': 300000,
  'salary_min': 130000,
  'site': 'https://hh.ru/search/vacancy/'},
 {'currency': 'руб.',
  'link': 'https://kaliningrad.hh.ru/vacancy/35275243?query=python',
  'name': 'DevOps - разработчик ПО',
  'salary_max'

  'site': 'https://hh.ru/search/vacancy/'},
 {'currency': 'USD',
  'link': 'https://kaliningrad.hh.ru/vacancy/35634568?query=python',
  'name': 'Реверс-инженер',
  'salary_max': 5000,
  'salary_min': 1500,
  'site': 'https://hh.ru/search/vacancy/'},
 {'currency': 'USD',
  'link': 'https://kaliningrad.hh.ru/vacancy/36842009?query=python',
  'name': 'Senior/Regular QA Manual Engineer',
  'salary_max': 2500,
  'salary_min': 1500,
  'site': 'https://hh.ru/search/vacancy/'},
 {'currency': None,
  'link': 'https://kaliningrad.hh.ru/vacancy/36316529?query=python',
  'name': 'Биоинформатик',
  'salary_max': None,
  'salary_min': None,
  'site': 'https://hh.ru/search/vacancy/'},
 {'currency': None,
  'link': 'https://kaliningrad.hh.ru/vacancy/36458238?query=python',
  'name': 'C/Go Developer',
  'salary_max': None,
  'salary_min': None,
  'site': 'https://hh.ru/search/vacancy/'},
 {'currency': None,
  'link': 'https://kaliningrad.hh.ru/vacancy/36775660?query=python',
  'name': 'Actuarial busine

  'name': 'Преподаватель IT дисциплин',
  'salary_max': 60000,
  'salary_min': None,
  'site': 'https://hh.ru/search/vacancy/'},
 {'currency': None,
  'link': 'https://kaliningrad.hh.ru/vacancy/37029897?query=python',
  'name': 'Compact Models Developer',
  'salary_max': None,
  'salary_min': None,
  'site': 'https://hh.ru/search/vacancy/'},
 {'currency': None,
  'link': 'https://kaliningrad.hh.ru/vacancy/36858333?query=python',
  'name': 'Специалист технической поддержки',
  'salary_max': None,
  'salary_min': None,
  'site': 'https://hh.ru/search/vacancy/'},
 {'currency': None,
  'link': 'https://kaliningrad.hh.ru/vacancy/36919975?query=python',
  'name': 'Ведущий эксперт управления разработкой / Team Lead (NSK)',
  'salary_max': None,
  'salary_min': None,
  'site': 'https://hh.ru/search/vacancy/'},
 {'currency': 'руб.',
  'link': 'https://kaliningrad.hh.ru/vacancy/36833137?query=python',
  'name': 'Системный администратор / DevOps / SRE',
  'salary_max': 160000,
  'salary_min': 140

...

  'name': 'Golang/ Go разработчик',
  'salary_max': None,
  'salary_min': None,
  'site': 'https://hh.ru/search/vacancy/'},
 {'currency': None,
  'link': 'https://kaliningrad.hh.ru/vacancy/36584253?query=python',
  'name': 'Go/Python разработчик (Tools DevOps team)',
  'salary_max': None,
  'salary_min': None,
  'site': 'https://hh.ru/search/vacancy/'},
 {'currency': None,
  'link': 'https://kaliningrad.hh.ru/vacancy/36782564?query=python',
  'name': 'Аналитик в креативный отдел / Creative Analyst (Middle)',
  'salary_max': None,
  'salary_min': None,
  'site': 'https://hh.ru/search/vacancy/'},
 {'currency': None,
  'link': 'https://kaliningrad.hh.ru/vacancy/36281690?query=python',
  'name': 'Senior Data scientist (ML модели корпоративного бизнеса)',
  'salary_max': None,
  'salary_min': None,
  'site': 'https://hh.ru/search/vacancy/'},
 {'currency': None,
  'link': 'https://kaliningrad.hh.ru/vacancy/36593364?query=python',
  'name': 'Разработчик Python',
  'salary_max': None,
  'salar

  'site': 'https://hh.ru/search/vacancy/'},
 {'currency': None,
  'link': 'https://kaliningrad.hh.ru/vacancy/36709026?query=python',
  'name': 'Младший количественный аналитик (стажер)',
  'salary_max': None,
  'salary_min': None,
  'site': 'https://hh.ru/search/vacancy/'},
 {'currency': 'руб.',
  'link': 'https://kaliningrad.hh.ru/vacancy/36708632?query=python',
  'name': 'Lead QA',
  'salary_max': None,
  'salary_min': 80000,
  'site': 'https://hh.ru/search/vacancy/'},
 {'currency': 'руб.',
  'link': 'https://kaliningrad.hh.ru/vacancy/35629377?query=python',
  'name': 'Системный администратор / DevOps',
  'salary_max': None,
  'salary_min': 80000,
  'site': 'https://hh.ru/search/vacancy/'},
 {'currency': 'руб.',
  'link': 'https://kaliningrad.hh.ru/vacancy/32526391?query=python',
  'name': 'Системный администратор Linux',
  'salary_max': 180000,
  'salary_min': None,
  'site': 'https://hh.ru/search/vacancy/'},
 {'currency': None,
  'link': 'https://kaliningrad.hh.ru/vacancy/37059750?

  'salary_min': None,
  'site': 'https://hh.ru/search/vacancy/'},
 {'currency': 'руб.',
  'link': 'https://kaliningrad.hh.ru/vacancy/36782660?query=python',
  'name': 'Backend разработчик C++/Qt',
  'salary_max': 250000,
  'salary_min': 200000,
  'site': 'https://hh.ru/search/vacancy/'},
 {'currency': None,
  'link': 'https://kaliningrad.hh.ru/vacancy/36340621?query=python',
  'name': 'Старший инженер по эксплуатации пакетных сетей',
  'salary_max': None,
  'salary_min': None,
  'site': 'https://hh.ru/search/vacancy/'},
 {'currency': None,
  'link': 'https://kaliningrad.hh.ru/vacancy/36733465?query=python',
  'name': 'Старший инженер по эксплуатации пакетных сетей',
  'salary_max': None,
  'salary_min': None,
  'site': 'https://hh.ru/search/vacancy/'},
 {'currency': None,
  'link': 'https://kaliningrad.hh.ru/vacancy/35995809?query=python',
  'name': 'Продуктовый аналитик в команду WebOffice',
  'salary_max': None,
  'salary_min': None,
  'site': 'https://hh.ru/search/vacancy/'},
 {'cur

 {'currency': 'руб.',
  'link': 'https://kaliningrad.hh.ru/vacancy/37036149?query=python',
  'name': 'Frontend/FullStack Developer (ReactJS/NodeJS)',
  'salary_max': 180000,
  'salary_min': 130000,
  'site': 'https://hh.ru/search/vacancy/'},
 {'currency': 'руб.',
  'link': 'https://kaliningrad.hh.ru/vacancy/29679783?query=python',
  'name': 'Инженер-программист',
  'salary_max': None,
  'salary_min': 70000,
  'site': 'https://hh.ru/search/vacancy/'},
 {'currency': None,
  'link': 'https://kaliningrad.hh.ru/vacancy/34006427?query=python',
  'name': 'Senior C/C++ Developer',
  'salary_max': None,
  'salary_min': None,
  'site': 'https://hh.ru/search/vacancy/'},
 {'currency': None,
  'link': 'https://kaliningrad.hh.ru/vacancy/37037809?query=python',
  'name': 'Web-программист - администратор сайта',
  'salary_max': None,
  'salary_min': None,
  'site': 'https://hh.ru/search/vacancy/'},
 {'currency': None,
  'link': 'https://kaliningrad.hh.ru/vacancy/34983004?query=python',
  'name': 'Спец

 {'currency': 'руб.',
  'link': 'https://kaliningrad.hh.ru/vacancy/35846701?query=python',
  'name': 'Backend Go Developer (Blue Team)',
  'salary_max': None,
  'salary_min': 200000,
  'site': 'https://hh.ru/search/vacancy/'},
 {'currency': 'руб.',
  'link': 'https://kaliningrad.hh.ru/vacancy/37038921?query=python',
  'name': 'Прикладной программист Linux',
  'salary_max': 130000,
  'salary_min': 80000,
  'site': 'https://hh.ru/search/vacancy/'},
 {'currency': None,
  'link': 'https://kaliningrad.hh.ru/vacancy/29323686?query=python',
  'name': 'Инженер DevOps',
  'salary_max': None,
  'salary_min': None,
  'site': 'https://hh.ru/search/vacancy/'},
 {'currency': None,
  'link': 'https://kaliningrad.hh.ru/vacancy/29323707?query=python',
  'name': 'Системный инженер Linux',
  'salary_max': None,
  'salary_min': None,
  'site': 'https://hh.ru/search/vacancy/'},
 {'currency': 'руб.',
  'link': 'https://kaliningrad.hh.ru/vacancy/37038930?query=python',
  'name': 'Разработчик FPGA (ПЛИС) Juni

In [24]:
with open("vac_sj.json", "w") as write_file:
    json.dump(vac_data_js, write_file)

In [25]:
with open("vac_hh.json", "w") as write_file:
    json.dump(vac_data, write_file)